### Shelve
https://docs.python.org/3/library/shelve.html#module-shelve

`shelve` is a simple persistent storage option for Python objects, acting like a dictionary but stored on disk.

In [1]:
import shelve
import requests

# Fetching data from an API and storing it using shelve
response = requests.get('https://example.com')

with shelve.open('myTest.db') as db:
    db['example_response'] = response

# Later access the data without needing to fetch it again
with shelve.open('myTest.db') as db:
    stored_response = db['example_response']

-----------
### Partial
https://docs.python.org/3/library/functools.html#functools.partial

`functools.partial` allows you to fix a certain number of arguments of a function and returns partial object that behaves like a function.

In [2]:
from functools import partial

from requests import Session

# Create session
session = Session()

graphql_url = 'https://countries.trevorblades.com/'

# creates function `post` that is calling session.post with url=graphql_url everytime
post = partial(session.post, url=graphql_url)

body = """
query {
    continents{
        name
    }
}
"""

resp = post(json={"query": body})
print("List of continents: ", resp.json(), sep="\n", end="\n\n")

body = """
query {
  country (code: "PL") {
    name,
    awsRegion
  }
}
"""

resp = post(json={"query": body})
print("AWS region of Poland: ", resp.json(), sep="\n")

List of continents: 
{'data': {'continents': [{'name': 'Africa'}, {'name': 'Antarctica'}, {'name': 'Asia'}, {'name': 'Europe'}, {'name': 'North America'}, {'name': 'Oceania'}, {'name': 'South America'}]}}

AWS region of Poland: 
{'data': {'country': {'name': 'Poland', 'awsRegion': 'eu-north-1'}}}


-------------
### Batched
https://docs.python.org/3/library/itertools.html#itertools.batched

`itertools.batched` simplifies the task of dividing iterables into batches of a specified size, offering a convenient solution for efficient batch processing.

In [3]:
try:
    # for python3.12
    from itertools import batched
except ImportError:
    # for older versions.
    # you need to install package with `pip install more_itertools`
    from more_itertools import batched

from concurrent.futures import ThreadPoolExecutor
from time import sleep, time
from timeit import Timer

DATA = [1] * 20000
MAX_WORKERS = 50
BATCH_SIZE = 400
SLEEP_TIME = 0.001
TIMEIT_REPEAT = 10

def simulate_io_operation(*args, **kwargs):
    sleep(SLEEP_TIME)

def process_data(item):
    simulate_io_operation(item)

def process_data_in_batch(items):
    for item in items:
        simulate_io_operation(item)


def test_single():
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = executor.map(process_data, DATA)


def test_batch():
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        results = executor.map(process_data_in_batch, batched(DATA, BATCH_SIZE))
        
print(
    f"Number of elements in list: {len(DATA)} | "
    f"batch size: {BATCH_SIZE} | "
    f"max workers: {MAX_WORKERS} | "
    f"times repeated: {TIMEIT_REPEAT}"
)

time_of_single = Timer(test_single).timeit(TIMEIT_REPEAT)
print(f"One thread per item ran {TIMEIT_REPEAT} times: {time_of_single} seconds")

time_of_batch = Timer(test_batch).timeit(TIMEIT_REPEAT)
print(f"One thread per {BATCH_SIZE} items ran {TIMEIT_REPEAT} times: {time_of_batch} seconds")

Number of elements in list: 20000 | batch size: 400 | max workers: 50 | times repeated: 10
One thread per item ran 10 times: 5.642835833015852 seconds
One thread per 400 items ran 10 times: 5.082546915975399 seconds


-------------------
### Single Dispatch
https://docs.python.org/3/library/functools.html#functools.singledispatch

`functools.singledispatch` enables you to create generic functions that behave differently based on the type of their first argument.



In [4]:
from functools import singledispatch

@singledispatch
def process(value):
    raise NotImplementedError("Unsupported type")

@process.register
def _(value: int):
    return value + 10

@process.register
def _(value: str):
    return value.upper()

# Usage
print(process(10))  # Output: 20

print(process('hello'))  # Output: 'HELLO'

try:
    process(['lama'])
except NotImplementedError as err:
    print(f"NotImplementedError error caught: {err}")

20
HELLO
NotImplementedError error caught: Unsupported type


------------
### Closing
https://docs.python.org/3/library/contextlib.html#contextlib.closing

`contextlib.closing` serves as a utility to ensure that resources are properly released after their use is complete.

In [5]:
import sqlite3
from contextlib import closing

with sqlite3.connect('myOtherTest') as connection:
    with closing(connection.cursor()) as cursor:
        cursor.execute("SELECT 'HELLO WORLD'")
        print(cursor.fetchall())



[('HELLO WORLD',)]
